In [1]:
import os
import pandas as pd
from datetime import datetime, timedelta, date, time
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [2]:
# Parameters
ssaa_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec/ssaa"
ssaa_path_df = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec/ssaa_c2.pkl"
perd_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec/perdidas102024.pkl"
spot_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec\\spot_elec.parquet"
spot_formated_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec/spot_h_periodos.pkl"
futures_path = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\processed\\elec\\futuros_elec.parquet"
path_omie_matrix = "c:\\Users\\mikel.perez\\0_Python_Projects - copia\\0_Python_projects_v2\\data\\raw\\elec\\matrix_ver.pkl"

In [3]:
# Parameters
ssaa_path_df = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/ssaa_c2.pkl"
perd_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/perdidas.pkl"
spot_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/spot_elec.parquet"
spot_formated_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/spot_h_periodos.pkl"
futures_path = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/processed/elec/futuros_elec.parquet"
path_omie_matrix = "/Users/mikelperez/0_Python_projects_v3_reduced/0_Python_projects_reduced/data/verification_project/elec/omie_matrix.pkl"


In [4]:
df_ssaa = pd.read_pickle(ssaa_path_df)
df_ssaa = df_ssaa.copy()
df_ssaa.loc[:, 'dup'] = df_ssaa.groupby('datetime').cumcount()
date_max = df_ssaa['datetime'].max()

df_spot = pd.read_pickle(spot_formated_path)
# df_spot_fil = df_spot[(df_spot['year'] == year) & (df_spot['month'] == month)]
df_spot_fil = df_spot[df_spot['datetime'] <= date_max]
df_spot_fil.reset_index(drop=True, inplace=True)
df_spot_fil.reset_index(drop=False, inplace=True)
df_spot_fil = df_spot_fil.copy()
df_spot_fil.loc[:, 'dup'] = df_spot_fil.groupby('datetime').cumcount()

df_merge = pd.merge(df_spot_fil, df_ssaa[['datetime', 'dup', 'ssaa']], on=['datetime', 'dup'], how='left')
df_merge.drop('index', axis=1, inplace=True)
df_merge.groupby('periodo')['ssaa'].mean()
df_merge = df_merge[['datetime', 'periodo', 'value', 'ssaa', 'dup']]
df_merge

,datetime,periodo,value,ssaa,dup
0,2014-01-01 00:00:00,6,20.02,NaN,0
1,2014-01-01 01:00:00,6,10.34,NaN,0
2,2014-01-01 02:00:00,6,5.35,NaN,0
3,2014-01-01 03:00:00,6,5.00,NaN,0
4,2014-01-01 04:00:00,6,0.50,NaN,0
...,...,...,...,...,...
100045,2025-05-31 19:00:00,6,31.78,17.172741,0
100046,2025-05-31 20:00:00,6,62.00,15.426091,0
100047,2025-05-31 21:00:00,6,64.31,16.026816,0
100048,2025-05-31 22:00:00,6,74.44,14.124158,0


In [5]:
df_merge_fil = df_merge[df_merge['datetime'].dt.year == 2024]
# df_merge_fil[['datetime', 'periodo', 'value', 'ssaa']].to_parquet('Spot y SSAA feb2018-may2025.parquet', index=False)

df_merge_fil.groupby('periodo')['value'].mean()

periodo
1    84.051525
2    76.608229
3    75.982142
4    55.630646
5    37.538721
6    58.619492
Name: value, dtype: float64

In [6]:
df_spot

,datetime,value,date,month,year,time,periodo
0,2014-01-01 00:00:00,20.02,2014-01-01,1,2014,00:00:00,6
1,2014-01-01 01:00:00,10.34,2014-01-01,1,2014,01:00:00,6
2,2014-01-01 02:00:00,5.35,2014-01-01,1,2014,02:00:00,6
3,2014-01-01 03:00:00,5.00,2014-01-01,1,2014,03:00:00,6
4,2014-01-01 04:00:00,0.50,2014-01-01,1,2014,04:00:00,6
...,...,...,...,...,...,...,...
101125,2025-07-15 19:00:00,72.84,2025-07-15,7,2025,19:00:00,1
101126,2025-07-15 20:00:00,112.92,2025-07-15,7,2025,20:00:00,1
101127,2025-07-15 21:00:00,135.87,2025-07-15,7,2025,21:00:00,1
101128,2025-07-15 22:00:00,135.05,2025-07-15,7,2025,22:00:00,2


In [7]:
df_merge.tail(40)

,datetime,periodo,value,ssaa,dup
100010,2025-05-30 08:00:00,5,53.01,14.569498,0
100011,2025-05-30 09:00:00,4,29.13,14.973369,0
100012,2025-05-30 10:00:00,4,1.83,16.686446,0
100013,2025-05-30 11:00:00,4,0.00,16.682774,0
100014,2025-05-30 12:00:00,4,-0.01,16.427392,0
100015,2025-05-30 13:00:00,4,-0.01,16.252940,0
100016,2025-05-30 14:00:00,5,-0.07,16.179634,0
100017,2025-05-30 15:00:00,5,-0.07,17.122708,0
100018,2025-05-30 16:00:00,5,-0.01,16.597339,0
100019,2025-05-30 17:00:00,5,5.00,17.559246,0


In [8]:
df_merge[['datetime', 'periodo', 'value', 'ssaa']]
df_merge.to_parquet('ssaa_precio.parquet')

In [9]:
df_perd = pd.read_pickle(perd_path)
df_perd = df_perd.copy()
df_perd.loc[:, 'dup'] = df_perd.groupby('datetime').cumcount()
df_perd.tail()

,datetime,20_td,30_td,61_td,62_td,63_td,64_td,dup
21162,2025-05-31 19:00:00,0.180,0.180,0.077,0.054,0.044,0.017,0
21163,2025-05-31 20:00:00,0.177,0.177,0.076,0.053,0.043,0.017,0
21164,2025-05-31 21:00:00,0.179,0.179,0.077,0.054,0.044,0.017,0
21165,2025-05-31 22:00:00,0.182,0.182,0.078,0.055,0.044,0.017,0
21166,2025-05-31 23:00:00,0.185,0.185,0.079,0.055,0.045,0.017,0


In [10]:
df_matrix = pd.merge(df_perd, df_merge, on=['datetime', 'dup'])
df_matrix.drop('dup', axis=1, inplace=True)
df_matrix = df_matrix[['datetime', 'periodo', 'value', 'ssaa', '20_td', '30_td', '61_td', '62_td', '63_td', '64_td']]
df_matrix.to_pickle(path_omie_matrix)

In [11]:
df_matrix

,datetime,periodo,value,ssaa,20_td,30_td,61_td,62_td,63_td,64_td
0,2023-01-01 00:00:00,6,0.00,26.761640,0.237,0.237,0.101,0.071,0.058,0.022
1,2023-01-01 01:00:00,6,0.00,27.891910,0.247,0.247,0.105,0.074,0.060,0.023
2,2023-01-01 02:00:00,6,0.00,38.725701,0.261,0.261,0.112,0.078,0.064,0.025
3,2023-01-01 03:00:00,6,0.00,38.365772,0.272,0.272,0.116,0.081,0.066,0.026
4,2023-01-01 04:00:00,6,0.00,40.265236,0.278,0.278,0.119,0.083,0.068,0.026
...,...,...,...,...,...,...,...,...,...,...
21161,2025-05-31 19:00:00,6,31.78,17.172741,0.180,0.180,0.077,0.054,0.044,0.017
21162,2025-05-31 20:00:00,6,62.00,15.426091,0.177,0.177,0.076,0.053,0.043,0.017
21163,2025-05-31 21:00:00,6,64.31,16.026816,0.179,0.179,0.077,0.054,0.044,0.017
21164,2025-05-31 22:00:00,6,74.44,14.124158,0.182,0.182,0.078,0.055,0.044,0.017


In [12]:
df_matrix_fil = df_matrix[df_matrix['datetime'].dt.year == 2024]

df_matrix_fil.groupby('periodo')['61_td'].mean()

periodo
1    0.072372
2    0.074269
3    0.063174
4    0.060737
5    0.048727
6    0.083650
Name: 61_td, dtype: float64